#**PROYECTO FINAL E-COMMERCE ANALYTICS**

#Grupo 4

##Emilio Palacín G.
##Oscar Espinoza
##Javier Ballén
##José Navarro

#Tratamiento de nulos en el DataFrame.

##- Objetivo:
    1. Imputar los valores faltantes de la columna 'Edad' usando la mediana de esa columna.
    2. Eliminar filas que no tengan texto en la columna 'Texto_Resena' (importante para tareas de NLP).

##- Efectos:
    - Modifica la columna 'Edad' rellenando NaN con la mediana actual de la columna.
    - Elimina filas del DataFrame que tengan NaN en 'Texto_Resena'.

##- Consideraciones:
    - Se asume que `df` es un pandas.DataFrame y contiene las columnas `Edad` y `Texto_Resena`.
    - La imputación con la mediana preserva la escala central de la variable numérica.
    - La eliminación de filas es crucial antes de convertir texto a vectores (p. ej. CountVectorizer).
    - El uso de `inplace=True` en `dropna` modifica `df` en sitio; si se requiere preservar el original, crear antes una copia.



In [ ]:
import pandas as pd
df = pd.read_csv('/content/dataset_ecommerce_moda.csv')

df['Edad'] = df['Edad'].fillna(df['Edad'].median())
df.dropna(subset=['Texto_Resena'], inplace=True)
print(df.head)

#Transformación de fechas en el DataFrame.

##- Objetivo:
    1. Convertir la columna 'Fecha_Compra' de texto/objeto a tipo datetime de pandas.
    2. Crear una nueva variable derivada 'Mes' extrayendo el mes de cada fecha de compra.

##- Efectos:
    - La columna 'Fecha_Compra' pasa de tipo object/string a datetime64.
    - Se añade una nueva columna 'Mes' con valores enteros del 1 al 12.

##- Consideraciones:
    - Se asume que `df` es un pandas.DataFrame y contiene la columna `Fecha_Compra`.
    - pd.to_datetime() intentará inferir el formato de fecha automáticamente.
    - Si hay fechas inválidas, se convertirán a NaT (Not a Time).
    - La columna 'Mes' es útil para análisis estacionales y agregaciones temporales.


#Estadistica y Vizualizacion

## Gráfico 1: Distribución de Edades

In [ ]:
df['Fecha_Compra'] = pd.to_datetime(df['Fecha_Compra'])
df['Mes'] = df['Fecha_Compra'].dt.month # Nueva variable derivada
print(df.head)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.histplot(df['Edad'], bins=20, kde=True, color='green')
plt.title('Distribución de Edad de los Clientes')
plt.savefig('grafico_edad.png') # Guarda la imagen
plt.show()

## Gráfico 2: Calificaciones por Categoría

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='Categoria', y='Calificacion', hue='Categoria', data=df, palette='viridis', legend=False)
plt.title('Distribución de Calificaciones por Categoría')
plt.savefig('grafico_boxplot.png')
plt.show()

<h1 align="center"> IA Aplicada / NLP </h1>

## Importamos la librería necesaria para stop words en español

In [ ]:
import nltk
from nltk.corpus import stopwords

## Descargamos el paquete de palabras vacías si no está presente

In [ ]:
nltk.download('stopwords')
stop_words_es = stopwords.words('spanish')

## Optimizamos el Vectorizador con el idioma correcto
## Agregamos 'max_df' para eliminar palabras que aparecen en casi todos los textos (ruido)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    stop_words=stop_words_es,
    max_features=50,
    max_df=0.95,  # Ignora palabras que aparecen en más del 95% de las reseñas
    min_df=2      # Ignora palabras que solo aparecen una vez
)

X_text = vectorizer.fit_transform(df['Texto_Resena'].astype(str)).toarray()
df_text = pd.DataFrame(X_text, columns=vectorizer.get_feature_names_out())
df_text.index = df.index

print("Vocabulario detectado (Top 50 en español):", vectorizer.get_feature_names_out())

#Vectorización de texto mediante Bag of Words (BoW).

##- Objetivo:
1. Convertir la columna de texto 'Texto_Resena' en una representación numérica
   usando el modelo Bag of Words.
2. Crear un DataFrame con las características de texto para unirlo al conjunto de datos principal.
##- Efectos:-
Se crea un objeto `vectorizer` de tipo CountVectorizer configurado para:
    * Eliminar stop words en inglés.
    * Limitar a las 50 palabras más frecuentes (max_features=50).
- Se genera una matriz dispersa convertida a array denso (`X_text`).
- Se crea `df_text`, un DataFrame donde cada columna representa una palabra del vocabulario.
- Se alinean los índices de `df_text` con los de `df` para permitir concatenación posterior.
 ##Consideraciones:-

 Se asume que `df` es un pandas.DataFrame y contiene la columna `Texto_Resena`.
- El parámetro `stop_words='english'` elimina palabras comunes que no aportan significado.
- `max_features=50` limita el vocabulario para reducir dimensionalidad y ruido.
- La conversión a `.toarray()` puede consumir memoria en datasets grandes.
- Es necesario alinear índices si se eliminaron filas previamente (p. ej. en el tratamiento de nulos).


In [ ]:
vectorizer = CountVectorizer(stop_words='english', max_features=50) # Top 50 palabras
X_text = vectorizer.fit_transform(df['Texto_Resena']).toarray()
# =============================================================================
df_text = pd.DataFrame(X_text, columns=vectorizer.get_feature_names_out())
df_text.index = df.index # Alinear índices

#Machine Learning Supervisado

##Objetivo: Predecir si es 'Recomendado' (1) o no (0)
##Variables predictoras: Edad, Precio, Calificación + Características del Texto (NLP)

In [ ]:
X_num = df[['Edad', 'Precio', 'Calificacion']]
X = pd.concat([X_num, df_text], axis=1) # Unimos datos numéricos + NLP
y = df['Recomendado']

## División 60% Train, 20% Val, 20% Test (Aprox usando train_test_split dos veces)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## Modelo: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

#Evaluacion

#Visualización de la Matriz de Confusión.

##- Objetivo:
Mostrar gráficamente el rendimiento del modelo de clasificación comparando las etiquetas reales (y_test) con las predicciones (y_pred).
##- Interpretación de la matriz:
- Diagonal principal: Predicciones correctas (Verdaderos Positivos y Verdaderos Negativos).
- Fuera de la diagonal: Errores del modelo (Falsos Positivos y Falsos Negativos).

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = rf.predict(X_test)
print("\n--- REPORTE DE CLASIFICACIÓN (MODELO SUPERVISADO) ---")
print(classification_report(y_test, y_pred))


plt.figure(figsize=(6,5))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Greens')
plt.title('Matriz de Confusión')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.savefig('grafico_matriz.png')
plt.show()

#Machine Learnign No Supervisado



```
# Esto tiene formato de código
```

## Clustering: Agrupar clientes por Edad y Precio (Gasto)

In [ ]:
from sklearn.preprocessing import StandardScaler

X_cluster = df[['Edad', 'Precio']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

## K-Means con 3 clústeres

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(X_scaled)

#Visualización de Clústeres mediante gráfico de dispersión.

##- Objetivo:
    Mostrar gráficamente la segmentación de clientes realizada por el algoritmo K-Means,
    representando cada cliente según su Edad y Precio (gasto).

##- Interpretación:
    - Cada punto representa un cliente.
    - Los colores indican a qué clúster pertenece cada cliente (0, 1 o 2).
    - Permite identificar patrones de segmentación basados en edad y comportamiento de gasto.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Edad', y='Precio', hue='Cluster', data=df, palette='viridis')
plt.title('Segmentación de Clientes (K-Means)')
plt.savefig('grafico_cluster.png')
plt.show()

#Exportar para Power BI

In [ ]:
df.to_csv('datos_para_powerbi.csv', index=False)
print("Archivo 'datos_para_powerbi.csv' generado para cargar en el Dashboard.")